In [67]:
import gist

from PIL import Image
import imageio
from IPython.display import display, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Parameters

In [68]:
# parameters
zoom = 15
lat = 30.4131736
lon = -87.600026
pixels = 640
samples_per_location = 1
precision = 4
num_images = 4
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')

# connect to db
db = dbcon.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

# image dimensions
meters_per_px = imd.zoom_in_meters_per_pixel(zoom, lat)
image_size = meters_per_px*pixels ## in meters

print("Meters per pixel:", round(meters_per_px,2))
print("Image size:", round(image_size,2), "in meters")

DB Credentials from file
DB connected successfully!!!
Meters per pixel: 4.12
Image size: 2636.76 in meters


### Get filenames of all the files in Google Drive that correspond to Madrid

In [70]:
area_name = 'US3_'

fnames = ima.get_filenames_of_city(area_name, IMG_FOLDER)
fnames

['US3_50.60035545236654_-77.01106789789257_16_640.png',
 'US3_47.960067695662076_-96.63297439094842_16_640.png',
 'US3_47.960067695662076_-96.63297439094842_14_640.png',
 'US3_38.44102401068488_-96.04316984424929_13_640.png',
 'US3_50.60035545236654_-77.01106789789257_15_640.png',
 'US3_52.329191065496744_-110.44840826605895_13_640.png',
 'US3_50.60035545236654_-77.01106789789257_14_640.png',
 'US3_38.15860639483217_-118.03113718932458_13_640.png',
 'US3_47.960067695662076_-96.63297439094842_15_640.png',
 'US3_38.15860639483217_-118.03113718932458_16_640.png',
 'US3_52.329191065496744_-110.44840826605895_16_640.png',
 'US3_38.44102401068488_-96.04316984424929_16_640.png',
 'US3_38.15860639483217_-118.03113718932458_15_640.png',
 'US3_52.329191065496744_-110.44840826605895_15_640.png',
 'US3_38.44102401068488_-96.04316984424929_14_640.png',
 'US3_47.960067695662076_-96.63297439094842_13_640.png',
 'US3_38.44102401068488_-96.04316984424929_15_640.png',
 'US3_50.60035545236654_-77.0110678

### Get image info of all the files of area name provided above (filenames, images, np.array)

In [71]:
images_info = ima.load_images_from_gdrive(fnames, IMG_FOLDER)

In [73]:
## !!! THIS WILL OPEN AN INTERACTIVE WINDOW FOR LABELLING THE IMAGES!!!
## FIRST PUT IN YOUR NAME AND THEN GO AHEAD WITH LABELLING
images_info = ima.add_labels_to_image_info(images_info)

## Label images in Database

In [74]:
filename_upload_to_db = 'temp1.csv'

ima.save_labels_as_csv(images_info, IMG_FOLDER, 
                       output_name = filename_upload_to_db, 
                       label_multi_name = "label_multi_pw",
                       label_binary_name = "label_binary_pw")

In [75]:
dbcon.write_labels_from_csv_to_db(images_lib_col, IMG_FOLDER, filename_upload_to_db)

21 labels added to db!


### Check binary labels

In [76]:
label_name = 'label_binary_pw'
query = {label_name: '0'}

doc = images_lib_col.find(query)
for x in doc:
    print(x['filename'], "has label", x[label_name])

US1_52.20482183860124_-92.48891258998856_13_640.png has label 0
US1_52.20482183860124_-92.48891258998856_14_640.png has label 0
US1_52.20482183860124_-92.48891258998856_15_640.png has label 0
US1_52.20482183860124_-92.48891258998856_16_640.png has label 0
US1_33.01060691483008_-118.04101400786547_13_640.png has label 0
US2_41.24686281982255_-121.72391747542048_14_640.png has label 0
US2_41.24686281982255_-121.72391747542048_15_640.png has label 0
US2_41.24686281982255_-121.72391747542048_16_640.png has label 0
US2_41.76932454759587_-105.42877312370616_14_640.png has label 0
US2_41.76932454759587_-105.42877312370616_15_640.png has label 0
US2_41.76932454759587_-105.42877312370616_16_640.png has label 0
US3_50.60035545236654_-77.01106789789257_15_640.png has label 0
US3_50.60035545236654_-77.01106789789257_16_640.png has label 0
US3_38.15860639483217_-118.03113718932458_13_640.png has label 0
US3_38.15860639483217_-118.03113718932458_15_640.png has label 0
US3_38.15860639483217_-118.0311

In [77]:
label_name = 'label_binary_pw'
query = {label_name: '1'}

doc = images_lib_col.find(query)
for x in doc:
    print(x['filename'], "has label", x[label_name])

US1_45.21160965529817_-93.54554509042913_13_640.png has label 1
US1_45.21160965529817_-93.54554509042913_14_640.png has label 1
US1_45.21160965529817_-93.54554509042913_15_640.png has label 1
US1_45.21160965529817_-93.54554509042913_16_640.png has label 1
US1_33.01060691483008_-118.04101400786547_14_640.png has label 1
US1_33.01060691483008_-118.04101400786547_15_640.png has label 1
US1_33.01060691483008_-118.04101400786547_16_640.png has label 1
US1_50.53186589235239_-108.35386180596005_13_640.png has label 1
US1_50.53186589235239_-108.35386180596005_14_640.png has label 1
US1_50.53186589235239_-108.35386180596005_15_640.png has label 1
US1_50.53186589235239_-108.35386180596005_16_640.png has label 1
US1_45.14093597874625_-101.4861640355236_13_640.png has label 1
US1_45.14093597874625_-101.4861640355236_14_640.png has label 1
US1_45.14093597874625_-101.4861640355236_15_640.png has label 1
US1_45.14093597874625_-101.4861640355236_16_640.png has label 1
US2_32.98997192625039_-84.9187594